In [1]:
useNLTK = True

import csv
import random
import math
import re
from collections import Counter
from pprint import pprint
from tqdm import tqdm
import pandas as pd
if useNLTK:
    from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
hc=pd.read_csv("HillaryClinton_tweets.csv")
jb=pd.read_csv("JebBush_tweets.csv")
tc=pd.read_csv("TedCruz_tweets.csv")
jc=pd.read_csv("DrJillStein_tweets.csv")
dt=pd.read_csv("realDonaldTrump_tweets.csv")
bs=pd.read_csv("BernieSanders_tweets.csv")

candidates = [hc,dt,jb,tc,jc,bs]
candidatesText = [x["text"] for x in candidates]
classes = ["Hillary Clinton","Donald Trump","Jeb Bush","Ted Cruz","Jill Stein","Bernie Sanders"]

In [3]:
def clean(txt):
    # see https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    txt = txt.lower()
    txt = re.sub(r'[^\w\s]','',txt)
    txt = re.sub(r"http\S+", "", txt)
    txt = re.sub(r"https\S+", "", txt)
    if useNLTK:
        lemmatizedList = []
        WNL = WordNetLemmatizer()
        for word in re.split("\W+", txt):
            lemmatizedList.append(WNL.lemmatize(word))
        txt = ' '.join(lemmatizedList)
    return txt

def tokenize(txt):
    return re.split("\W+", txt)

def countWords(txt):
    count = Counter(tokenize(txt))
    count = dict(count)
    try:
        del count[""]
    except KeyError:
        pass
    return dict([a, float(x)] for a, x in count.items())

In [4]:
#sklearn implementation

import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def buildDataFrame():
    d = []
    i = []
    for ind, candidate in tqdm(enumerate(candidates)):
        for rowInd, row in candidate.iterrows():
            d.append(row["text"])
        i.extend([classes[ind] for _ in candidate.itertuples()])
    return pd.DataFrame({"text":d, "class":i})

skdf = buildDataFrame()
countVectorizer = CountVectorizer()
counts = countVectorizer.fit_transform(skdf["text"].values)
classifier = MultinomialNB()
targets = skdf['class'].values
classifier.fit(counts, targets)

6it [00:02,  2.81it/s]


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
def train(method):
    PC = 1/len(classes)
    PXC = dict.fromkeys(classes)
    totalWords = dict.fromkeys(classes)
    if method == 0:
        totalDocs = dict.fromkeys(classes)
        for doc in skdf.itertuples():
            if PXC[doc[1]] == None:
                PXC[doc[1]] = dict()
            if totalDocs[doc[1]] == None:
                totalDocs[doc[1]] = 0
            for word in tokenize(doc[2]):
                try:
                    PXC[doc[1]][word] += 1
                except KeyError:
                    PXC[doc[1]][word] = 1
            totalDocs[doc[1]] += 1
        for candidate, words in PXC.items():
            for word, instances in words.items():
                PXC[candidate][word] = instances/totalDocs[candidate]
    elif method == 1:
        for ind, candidate in tqdm(enumerate(classes)):
            for tweet in candidatesText[ind]:
                tweet = clean(tweet)
                if PXC[candidate]==None:
                    PXC[candidate]=[]
                PXC[candidate].append(Counter(countWords(tweet)))
                if totalWords[candidate]==None:
                    totalWords[candidate]=0
                totalWords[candidate]+=len(tokenize(tweet))  
            PXC[candidate] = dict(sum(PXC[candidate],Counter()))
        for candidate in tqdm(classes):
            for word,i in PXC[candidate].items():
                PXC[candidate][word] = i/totalWords[candidate]
    else:
        raise TypeError
    
    for candidate in tqdm(classes):
        allWords = sum([list(PXC[x].keys()) for x in classes], [])
        allWordsLength = len(allWords)
        PX = list(dict(Counter(allWords)).items())
        PX = {i[0]:i[1]/allWordsLength for i in PX}
    return PC, PXC, PX

    

PC, PXC, PX = train(method=1)
print("Finished training")

6it [01:18, 13.12s/it]
100%|██████████| 6/6 [00:00<00:00, 21.46it/s]

Finished training


In [18]:
from math import e, log as ln

def predictClass(inp):
    inp = clean(inp)
    probabilities = dict.fromkeys(classes)
    for candidate in classes:
        if probabilities[candidate]==None:
            probabilities[candidate] = 0
        for word in tokenize(inp):
            try:
                probability = (PC*PXC[candidate][word])/PX[word]
            except KeyError:
                probability = 0
            try:
                probabilities[candidate] += ln(probability)
            except ValueError:
                pass
    for candidate, probability in probabilities.items():
        probabilities[candidate] = e**probability
    mostLikely = sorted(list(probabilities.items()), key=lambda x: x[1])[-1]
    if mostLikely[1]==0:
        return False
    else:
        #pprint(probabilities)
        return mostLikely

In [19]:
def validateModel(useTrainingSet=False):
    if useTrainingSet:
        validationSet = [pd.read_csv("HillaryClinton_tweets.csv"), \
            pd.read_csv("realDonaldTrump_tweets.csv"), pd.read_csv("JebBush_tweets.csv"), \
            pd.read_csv("TedCruz_tweets.csv"), pd.read_csv("DrJillStein_tweets.csv"), \
            pd.read_csv("BernieSanders_tweets.csv")]
    else:
        validationSet = [pd.read_csv("HillaryClinton_tweets NEW.csv"), \
            pd.read_csv("realDonaldTrump_tweets NEW.csv"), pd.read_csv("JebBush_tweets NEW.csv"), \
            pd.read_csv("TedCruz_tweets NEW.csv"), pd.read_csv("DrJillStein_tweets NEW.csv"), \
            pd.read_csv("BernieSanders_tweets NEW.csv")]
    
    validationSet = [x["text"] for x in validationSet]
    totalTweets = 0
    correctTweets = 0
    for ind, candidate in tqdm(enumerate(classes)):
        for validate in validationSet[ind]:
            totalTweets += 1
            try:
                if predictClass(validate)[0] == candidate:
                    correctTweets += 1
            except TypeError:
                pass
    return correctTweets/totalTweets

print(validateModel())
print(validateModel(useTrainingSet=True))

6it [00:00, 20.45it/s]
0it [00:00, ?it/s]

0.5072886297376094


6it [00:07,  1.24s/it]

0.46282614311547404


In [20]:
print(predictClass("Memes are responsible for climate change"))
print(predictClass("I love hispanics"))
print(predictClass("Billionaires suck"))
print(predictClass("Let us become a glorious socialist empire and serve the motherland"))
print(predictClass("Let's make America Great again!"))

('Bernie Sanders', 17.37172624455413)
('Donald Trump', 3.816016115301887)
('Bernie Sanders', 2.713632339923281)
('Jeb Bush', 1009.3878428802311)
('Donald Trump', 356.19054136576557)


In [21]:
examples = ["Let us become a glorious socialist empire and serve the motherland"]
example_counts = countVectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions[0]

'Hillary Clinton'

In [6]:
def validateModelSKLearn(useTrainingSet=False):
    if useTrainingSet:
        validationSet = [pd.read_csv("HillaryClinton_tweets.csv"), \
            pd.read_csv("realDonaldTrump_tweets.csv"), pd.read_csv("JebBush_tweets.csv"), \
            pd.read_csv("TedCruz_tweets.csv"), pd.read_csv("DrJillStein_tweets.csv"), \
            pd.read_csv("BernieSanders_tweets.csv")]
    else:
        validationSet = [pd.read_csv("HillaryClinton_tweets NEW.csv"), \
            pd.read_csv("realDonaldTrump_tweets NEW.csv"), pd.read_csv("JebBush_tweets NEW.csv"), \
            pd.read_csv("TedCruz_tweets NEW.csv"), pd.read_csv("DrJillStein_tweets NEW.csv"), \
            pd.read_csv("BernieSanders_tweets NEW.csv")]
    
    validationSet = [x["text"] for x in validationSet]
    totalTweets = 0
    correctTweets = 0
    for ind, candidate in tqdm(enumerate(classes)):
        for validate in validationSet[ind]:
            totalTweets += 1
            samples = [validate]
            sampleCounts = countVectorizer.transform(samples)
            predictions = classifier.predict(sampleCounts)
            prediction = predictions[0]
            try:
                if prediction == candidate:
                    correctTweets += 1
            except TypeError:
                pass
    return correctTweets/totalTweets

validateModelSKLearn(useTrainingSet=False)

6it [00:00,  7.98it/s]


0.749271137026239

In [ ]:
for i in skdf:
    print(i.loc(["class"]).value)
    raise IndexError